# **Pre Classification Analysis** 

In [1]:
import sys
sys.path
sys.path.append('c:\\Users\\3leso\\Documents\\Elena\\Uni\\MasterThesis\\MasterThesis_final\\code\\preprocessing')

### Set file paths

In [2]:
file_comments = "../../../../data/reddit/PoliticalDiscussion_comments.zst"
file_submissions = "../../../../data/reddit/PoliticalDiscussion_submissions.zst"

subreddit_here = "PoliticalDiscussion"


output_path = f"output/{subreddit_here}_new_preprocessing_com_rep.csv"



In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import matplotlib.pyplot as plt
import seaborn as sns
import community as community_louvain

import zstandard
import io
import zipfile

from sklearn.model_selection import train_test_split
from scipy import sparse
import random
from itertools import combinations
import collections
import multiprocessing
import copy
import time
import gurobipy
from gurobipy import *

pd.options.mode.chained_assignment = None

In [4]:
from preprocessing_functions import import_subreddit
from preprocessing_functions import filter_subreddit
from preprocessing_functions import aggregate_interactions_directed
from preprocessing_functions import get_counts_and_shares
from preprocessing_functions import find_max
from preprocessing_functions import build_undirected
from preprocessing_functions import get_submissions
from preprocessing_functions import get_comments_and_replies
from preprocessing_functions import assign_types
from preprocessing_functions import find_relevant_submissions
from preprocessing_functions import match_reply_comment_submission
from preprocessing_functions import preprocess_texts
from preprocessing_functions import import_merge_predicted_labels

***

## **Data Import**

In [5]:
# write comments to parquet
#import_subreddit_to_parquet(file_comments, "../../../data/reddit/political_discussion_comments_parquet_new.zst", chunk_size=100_000, data_type="comments")

# write submissions to parquet
#import_subreddit_to_parquet(file_submissions, "../../../data/reddit/political_discussion_submissions_parquet_new.zst", chunk_size=100_000, data_type="submissions")


In [6]:
data_comments = pd.read_parquet("../../../../data/reddit/political_discussion_comments_parquet_new.zst", engine="pyarrow")
data_submissions = pd.read_parquet("../../../../data/reddit/political_discussion_submissions_parquet_new.zst", engine="pyarrow")


In [7]:
# import data
#data_comments = import_subreddit(file_comments)
#data_submissions = import_subreddit(file_submissions)


In [8]:
# make df
df_comments = pd.DataFrame(data_comments)
df_submissions = pd.DataFrame(data_submissions)

In [9]:
# submissions
# filter relevant columns

submissions = get_submissions(df_submissions)
submissions

,author,created_utc,id,num_comments,score,send_replies,stickied,subreddit,subreddit_id,title,name,likes,ups,category,selftext,type,date
0,[deleted],1.302798e+09,gq004,4,10,None,None,PoliticalDiscussion,t5_2sfmf,"John Paulson made $5,000,000,000 this year bec...",t3_gq004,None,21.0,None,,submission,2011-04-14
1,dtmike07,1.302996e+09,grqj9,19,7,None,None,PoliticalDiscussion,t5_2sfmf,The fundamental problem with politics (an essa...,t3_grqj9,None,15.0,None,I've been an adherent of or flirted with most ...,submission,2011-04-17
4,snorky94,1.303003e+09,grsi7,17,3,None,None,PoliticalDiscussion,t5_2sfmf,Can someone kindly explain this ridiculous bip...,t3_grsi7,None,10.0,None,"Oops, I mean ""bipartisanship.""",submission,2011-04-17
3,usdave,1.303065e+09,gs6ax,7,11,None,None,PoliticalDiscussion,t5_2sfmf,How to go about changing voting systems to fav...,t3_gs6ax,None,12.0,None,I have read about various alternative voting s...,submission,2011-04-17
2,soldier_physicist,1.303072e+09,gs8t2,17,12,None,None,PoliticalDiscussion,t5_2sfmf,Is anyone else thoroughly fed the fuck up?,t3_gs8t2,None,20.0,None,Taxation without representation was the main c...,submission,2011-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188499,No_Firefighter_5238,1.704044e+09,18vcntj,2,1,False,False,PoliticalDiscussion,t5_2sfmf,Do you agree with the author’s assessment of B...,t3_18vcntj,None,1.0,None,[removed],submission,2023-12-31
188500,pinner52,1.704045e+09,18vcxwl,20,1,True,False,PoliticalDiscussion,t5_2sfmf,Billionaire Chamath Palihapitiya praises Trump...,t3_18vcxwl,None,1.0,None,\r \nClip can be found here at 17 seconds: [h...,submission,2023-12-31
188501,twentyx2,1.704047e+09,18vdmvm,1,1,True,False,PoliticalDiscussion,t5_2sfmf,What explains the weird capital police videos ...,t3_18vdmvm,None,1.0,None,[removed],submission,2023-12-31
188502,tlm226,1.704060e+09,18vi9hi,1,1,True,False,PoliticalDiscussion,t5_2sfmf,My predictions for 2024,t3_18vi9hi,None,1.0,None,[removed],submission,2023-12-31


In [10]:
# comments
# select relevant columns only

df_comments_replies = get_comments_and_replies(df_comments)
df_comments_replies

,author,created_utc,parent_id,subreddit_id,body,link_id,name,id,ups,score,downs,subreddit,likes,replies,stickied,send_replies,comment_type,date
0,[deleted],1.302878e+09,gq004,2sfmf,What do tax breaks have to do with the profits...,gq004,c1pk4gc,c1pk4gc,1,1,0,PoliticalDiscussion,None,None,None,None,None,2011-04-15
1,[deleted],1.303010e+09,grsi7,2sfmf,"I'll let Lewis Black explain it to you. ""The D...",grsi7,c1pt5h6,c1pt5h6,8,8,0,PoliticalDiscussion,None,None,None,None,None,2011-04-17
2,snorky94,1.303012e+09,c1pt5h6,2sfmf,Well that actually kind of helps a lot.,grsi7,c1ptav6,c1ptav6,1,1,0,PoliticalDiscussion,None,None,None,None,None,2011-04-17
3,[deleted],1.303014e+09,grsi7,2sfmf,"Well, to be frank, it sounds like you are fair...",grsi7,c1ptgcy,c1ptgcy,6,6,0,PoliticalDiscussion,None,None,None,None,None,2011-04-17
4,snorky94,1.303017e+09,c1ptgcy,2sfmf,"No, I am honestly open to any explanation of w...",grsi7,c1pto8a,c1pto8a,2,2,0,PoliticalDiscussion,None,None,None,None,None,2011-04-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6839333,Remarkable-Code-3237,1.704066e+09,kfkre62,2sfmf,The attacks on Trump in 2016 did not work eith...,18tqr4m,kfrm5gn,kfrm5gn,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01
6839334,stewartm0205,1.704066e+09,kfa350f,2sfmf,Not many decades. I fully expect Texas and Flo...,18s2to7,kfrmvtn,kfrmvtn,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01
6839335,intersexy911,1.704067e+09,kfofh9w,2sfmf,Gore won Florida in 2000. The Supreme Court st...,18uynd7,kfrnx9b,kfrnx9b,3,3,0,PoliticalDiscussion,None,,False,True,None,2024-01-01
6839336,Moleday1023,1.704067e+09,18uryo1,2sfmf,Fact checking politicians and their surrogates...,18uryo1,kfrnxrh,kfrnxrh,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01


In [11]:
# assign types and extract them to separate dfs
comments, replies = assign_types(df_comments_replies, submissions)

In [12]:
print("Comments: ", len(comments))
print("Replies:", len(replies))

Comments:  1371225
Replies: 1071170


## Plot 

* no preprocessing of tokens so far

In [13]:
# df_comments_replies["tokens"] = df_comments_replies["body"].apply(lambda x: x.split())
# df_comments_replies["word_count"] = df_comments_replies["tokens"].apply(len)

# print(df_comments_replies['word_count'].min())
# print(df_comments_replies['word_count'].max())
# print(df_comments_replies['word_count'].value_counts())

# fig, ax = plt.subplots()
# ax.hist(df_comments_replies['word_count'], bins = 20, range = (0, 600))
# ax.set_title("Histogram of Word Count")
# #ax.set_xticks([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]) 
# #ax.set_xticklabels(["0.0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1"])  
# ax.set_xlabel("Word Counts")
# ax.set_ylabel("Frequency")
# ax.grid(True)

### Filter relevant submissions

* relevant = above subreddit-average number of comments
* relevant_comments: those with a parent, that is in relevant_submissions

In [14]:
# select relevant submissions (by avg. number of comments)

relevant_submissions, relevant_comments = find_relevant_submissions(df_comments = comments, df_submissions = submissions, submission_id_col = "parent_id", comment_id_col = "id")


print(f"Number of relevant submissions: {len(relevant_submissions)}")
print("Lost submissions: ", len(submissions) - len(relevant_submissions))

relevant_submissions

relevant_comments.reset_index(drop=True, inplace=True)
relevant_comments

8.069921962358315
Number of relevant submissions: 26971
Lost submissions:  161533


,author,created_utc,parent_id,subreddit_id,body,link_id,name,id,ups,score,downs,subreddit,likes,replies,stickied,send_replies,comment_type,date,type
0,[deleted],1.303087e+09,gs8t2,2sfmf,I disagree. Our forefathers were truly taxed w...,gs8t2,c1pxqx6,c1pxqx6,3,3,0,PoliticalDiscussion,None,None,None,None,None,2011-04-18,comment
1,hackel,1.303089e+09,gs8t2,2sfmf,To compare the current situation to that of th...,gs8t2,c1pxw0n,c1pxw0n,5,5,0,PoliticalDiscussion,None,None,None,None,None,2011-04-18,comment
2,dtmike07,1.303098e+09,gs8t2,2sfmf,"I agree with hackel - we have representation, ...",gs8t2,c1pypou,c1pypou,4,4,0,PoliticalDiscussion,None,None,None,None,None,2011-04-18,comment
3,mahkato,1.304018e+09,gs8t2,2sfmf,Google Ron Paul. Come back refreshed.,gs8t2,c1rfv4z,c1rfv4z,-1,-1,0,PoliticalDiscussion,None,None,None,None,None,2011-04-28,comment
4,orbit1979,1.304471e+09,gs8t2,2sfmf,"""Taxation without representation...We have tha...",gs8t2,c1sb3lv,c1sb3lv,1,1,0,PoliticalDiscussion,None,None,None,None,None,2011-05-04,comment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900836,Tex-Rob,1.704065e+09,18tsmpm,2sfmf,Parties are terrible as a way to control outco...,18tsmpm,kfrj0ft,kfrj0ft,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01,comment
900837,Madhatter25224,1.704065e+09,18uryo1,2sfmf,Lobbying is illegal. Anyone elected to any pub...,18uryo1,kfrkedm,kfrkedm,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01,comment
900838,gaxxzz,1.704066e+09,18uou00,2sfmf,Russia has been threatening to use nuclear wea...,18uou00,kfrlee3,kfrlee3,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01,comment
900839,ruminaui,1.704066e+09,18uyiyk,2sfmf,"Because we all know who the nominee is, and no...",18uyiyk,kfrll89,kfrll89,1,1,0,PoliticalDiscussion,None,,False,True,None,2024-01-01,comment


### Build df
* Match first level replies to relevant comments to relevant submissions
* Join = 'inner', because only keep replies, that have a parent in the relevant_comments
    * Missings due to replies, that did not find a comment to belong to.
    * This happend, because comments were dropped, if they were a comment to a submission, that had too little comments (below average)
    * Between comments and submissions, there won't be missings, as relevant_submissions were used to filter relevant_comments 

In [15]:
df_left = match_reply_comment_submission(replies, relevant_comments, relevant_submissions, 'left')
df_inner = match_reply_comment_submission(replies, relevant_comments, relevant_submissions, 'inner') # correct join

print("Left Join:", len(df_left))
print("Inner Join: ", len(df_inner))

print("Length of children in left, that are in inner as well: ", len(df_left[~df_left['msg_id_child'].isin(df_inner['msg_id_child'])]))
print("Difference: ", len(df_left) - len(df_inner))

Left Join: 1017120
Inner Join:  819653
Length of children in left, that are in inner as well:  197467
Difference:  197467


In [16]:
df_matched = match_reply_comment_submission(replies, relevant_comments, relevant_submissions, 'inner')
df_matched

,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c1t5qvl,c1t7nc7,Exactly right. Apparently the founders valued ...,PoliticalDiscussion,1.304981e+09,[deleted],2,2,0,gs8t2,...,[deleted],3,3,0,1.304962e+09,Taxation without representation was the main c...,soldier_physicist,20.0,12,1.303072e+09
1,c1pxw0n,c1t97is,WE DO NOT HAVE DEMOCRACY \n\nOur current polit...,PoliticalDiscussion,1.304998e+09,thetimeisnow,3,3,0,gs8t2,...,hackel,5,5,0,1.303089e+09,Taxation without representation was the main c...,soldier_physicist,20.0,12,1.303072e+09
2,c1sb3lv,c1td750,&gt;Vote in November 2012.\n\ndon't encourage him,PoliticalDiscussion,1.305057e+09,[deleted],0,0,0,gs8t2,...,orbit1979,1,1,0,1.304471e+09,Taxation without representation was the main c...,soldier_physicist,20.0,12,1.303072e+09
3,c1ttks1,c1tvxf7,"thats a good post, thanks moezaly. be interest...",PoliticalDiscussion,1.305304e+09,ShmoopyGuy,2,2,0,h9wsz,...,moezaly,4,4,0,1.305263e+09,Muslims who want to live under Islamic Sharia ...,[deleted],6.0,5,1.305222e+09
4,c1trw1e,c1tvz52,"yeah i see exactly where your coming from, she...",PoliticalDiscussion,1.305305e+09,ShmoopyGuy,1,1,0,h9wsz,...,oddmanout,6,6,0,1.305242e+09,Muslims who want to live under Islamic Sharia ...,[deleted],6.0,5,1.305222e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819648,kfnmrvq,kfqrpy8,Note this is because originally women and mino...,PoliticalDiscussion,1.704054e+09,metarinka,31,31,0,18uynd7,...,Jet451,155,155,0,1.703996e+09,"The Census counts all people in the US, but Co...",tpthatsme,13.0,13,1.703995e+09
819649,kfpzmgq,kfquxpu,"You're not wrong, honestly. Democrats want to ...",PoliticalDiscussion,1.704055e+09,Taervon,3,3,0,18uyiyk,...,baxterstate,-2,-2,0,1.704044e+09,"Pence, Scott and Bergum have all refused to en...",finnthehumanmertins,133.0,133,1.703994e+09
819650,kfn8p5y,kfrcund,My idea would be to still have 2 Senators per ...,PoliticalDiscussion,1.704062e+09,blaqsupaman,1,1,0,18uryo1,...,WichitaTheOG,18,18,0,1.703990e+09,What are your favorite features or quirks of c...,19Krakatoa96,36.0,36,1.703976e+09
819651,kfn0ce4,kfrl1pz,>Putin has been *threatening* to use them for ...,PoliticalDiscussion,1.704066e+09,gaxxzz,1,1,0,18uou00,...,BitterFuture,120,120,0,1.703987e+09,Much has been made about Putin using a nuke to...,Mad_Prog_1,35.0,35,1.703968e+09


In [17]:
duplicated = df_matched[['body_child', "body_parent", 'subreddit', "msg_id_child", "msg_id_parent", "author_parent", "author_child"]]
duplicated = duplicated[duplicated.duplicated(['body_child', "body_parent", "author_parent", "author_child"], keep=False) == True]
duplicated

,body_child,body_parent,subreddit,msg_id_child,msg_id_parent,author_parent,author_child


### Filter for Subreddit

In [18]:
# filter for specific subreddit --> technically unnecessary
# also converts date-time and label column

df = filter_subreddit(df_matched, subreddit_here)


### Select High Quality Texts


* Filtering
    * remove deleted or removed parent and child texts (submissions no problem)
    * remove deleted authors of parents and childs (submissions no problem)
    * remove bodies (parents and childs) whose exact text occurred too often within the subreddit (i.e. bots)
    

* Preprocessing of texts
    * remove user mentions
    * remove subreddit mentions
    * remove URLs
    * remove line breaks
    * remove multiple spaces
    * replace &amp with and
    * (remove brackets and exclamation marks --> better not, they contain context)


* Filtering, second step
    * remove too short replies & comments (submissions no problem)


In [19]:
df['body_child'].value_counts().head(5)

#df[df['body_child'] == "Sorry, your post has been removed because your account does not currently meet the minimum required karma to post in this subreddit.\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AmericanPolitics) if you have any questions or concerns.*"]

body_child
[removed]                                                                          32880
[deleted]                                                                          27480
Why?                                                                                 188
Do not submit low investment content. This subreddit is for genuine discussion.      151
Yes                                                                                  116
Name: count, dtype: int64

In [20]:
df = preprocess_texts(data = df, length = 5)
df

Length 5 removes 38264 interactions


,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c1pxw0n,c1t97is,WE DO NOT HAVE DEMOCRACY Our current political...,PoliticalDiscussion,2011-05-10 03:34:29,thetimeisnow,3,3,0,gs8t2,...,hackel,5,5,0,2011-04-18 01:02:53,Taxation without representation was the main c...,soldier_physicist,20.0,12,2011-04-17 20:29:13
1,c1ttks1,c1tvxf7,"thats a good post, thanks moezaly. be interest...",PoliticalDiscussion,2011-05-13 16:31:01,ShmoopyGuy,2,2,0,h9wsz,...,moezaly,4,4,0,2011-05-13 05:03:57,Muslims who want to live under Islamic Sharia ...,[deleted],6.0,5,2011-05-12 17:34:38
2,c1trw1e,c1tvz52,"yeah i see exactly where your coming from, she...",PoliticalDiscussion,2011-05-13 16:38:18,ShmoopyGuy,1,1,0,h9wsz,...,oddmanout,6,6,0,2011-05-12 23:14:26,Muslims who want to live under Islamic Sharia ...,[deleted],6.0,5,2011-05-12 17:34:38
3,c1trw1e,c1uh01k,"I agree, with hopefully adding a slight correc...",PoliticalDiscussion,2011-05-17 04:55:14,eragonpool,2,2,0,h9wsz,...,oddmanout,6,6,0,2011-05-12 23:14:26,Muslims who want to live under Islamic Sharia ...,[deleted],6.0,5,2011-05-12 17:34:38
4,c1v1hf2,c1v2e0i,Sort of the same thing I was thinking. You wou...,PoliticalDiscussion,2011-05-20 10:41:58,HighAltitudeNoOpen,3,3,0,hfmwy,...,Denny_Craine,3,3,0,2011-05-20 05:11:03,individually. I promise I'm not illiterate. lq...,HighAltitudeNoOpen,14.0,11,2011-05-20 04:36:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643764,kfnmrvq,kfqrpy8,Note this is because originally women and mino...,PoliticalDiscussion,2023-12-31 20:22:12,metarinka,31,31,0,18uynd7,...,Jet451,155,155,0,2023-12-31 04:17:49,"The Census counts all people in the US, but Co...",tpthatsme,13.0,13,2023-12-31 03:53:53
643765,kfpzmgq,kfquxpu,"You're not wrong, honestly. Democrats want to ...",PoliticalDiscussion,2023-12-31 20:41:40,Taervon,3,3,0,18uyiyk,...,baxterstate,-2,-2,0,2023-12-31 17:31:42,"Pence, Scott and Bergum have all refused to en...",finnthehumanmertins,133.0,133,2023-12-31 03:47:32
643766,kfn8p5y,kfrcund,My idea would be to still have 2 Senators per ...,PoliticalDiscussion,2023-12-31 22:35:22,blaqsupaman,1,1,0,18uryo1,...,WichitaTheOG,18,18,0,2023-12-31 02:39:37,What are your favorite features or quirks of c...,19Krakatoa96,36.0,36,2023-12-30 22:41:24
643767,kfn0ce4,kfrl1pz,Putin has been threatening to use them for ove...,PoliticalDiscussion,2023-12-31 23:33:07,gaxxzz,1,1,0,18uou00,...,BitterFuture,120,120,0,2023-12-31 01:44:41,Much has been made about Putin using a nuke to...,Mad_Prog_1,35.0,35,2023-12-30 20:25:58


In [21]:
#df

duplicated = df[['body_child', "body_parent", 'subreddit', "msg_id_child", "msg_id_parent", "author_parent", "author_child"]]
duplicated = duplicated[duplicated.duplicated(['body_child', "body_parent", "author_parent", "author_child"], keep=False) == True]
duplicated


,body_child,body_parent,subreddit,msg_id_child,msg_id_parent,author_parent,author_child


### Save and export

In [22]:
# to csv
zip_filename = f"output/{subreddit_here}_new_preprocessing_com_rep.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
    with zf.open(output_path, 'w') as f:
        df.to_csv(f, index=False)


# import again
with zipfile.ZipFile(zip_filename, 'r') as zf:
    with zf.open(output_path) as f:
        df = pd.read_csv(f)

df

df.to_csv(output_path, index = False)

***
## **Descriptive Stats**

In [23]:
# get nr of users

comm_auth = list(df_comments['author'].unique())
sub_auth = list(df_submissions['author'].unique())

authors = comm_auth + sub_auth
print(len(authors))
authors = set(authors)
authors.remove('[deleted]')
print("Unique Users: ", len(set(authors)))

273272
Unique Users:  240187


### Time Stats and originally pulled counts

In [24]:
# Time frame
print("Time Frame \n---------------------------------------------")
print(f"Beginning of time frame: {df_comments_replies['date'].min()}")
print(f"End of time frame: {df_comments_replies['date'].max()}")
print("---------------------------------------------\n")

print("Number of Submissions, Comments and Replies \n---------------------------------------------")
print(f"Number of submissions: {len(submissions)}")
print(f"Number of comments and replies: {len(df_comments_replies)}")
print("---------------------------------------------\n")


# irrelevant, because submissions need to be filtered and matched to comments and replies

# print("Number of pure comments and first level replies \n---------------------------------------------")
# print(f"Number of comments: {len(comments)}")
# print(f"Number of replies: {len(replies)}")
# print(f"Sum of both: {len(comments) + len(replies)}")
# print("---------------------------------------------\n")

# print("Number of lost replies \n---------------------------------------------")
# print(f"Lost replies: {len(df_comments_replies) - (len(comments) + len(replies))}")
# print("---------------------------------------------\n")

Time Frame 
---------------------------------------------
Beginning of time frame: 2011-04-15
End of time frame: 2024-01-01
---------------------------------------------

Number of Submissions, Comments and Replies 
---------------------------------------------
Number of submissions: 188504
Number of comments and replies: 6839338
---------------------------------------------



### Descriptive Stats after Matching

In [25]:
# Descriptive stats after matching

print("Number of unique replies: ", len(df_matched['msg_id_child'].unique()))
print("Number of unique comments: ", len(df_matched['msg_id_parent'].unique()))
print("Number of unique submissions: ", len(df_matched['submission_id'].unique()))

Number of unique replies:  819649
Number of unique comments:  365333
Number of unique submissions:  26895


### Descriptive Stats after cleaning for high quality texts

In [26]:
print("Unique Replies: ", len(df['msg_id_child'].unique()))
print("Unique Comments: ", len(df['msg_id_parent'].unique()))
print("Unique Submissions: ", len(df['submission_id'].unique()))

Unique Replies:  643769
Unique Comments:  303610
Unique Submissions:  26635


In [27]:
print("Unique Users: ", len(set(df['author_child']).union(df['author_parent'], df['author_submission'])))

Unique Users:  100696
